Imports

In [220]:
import itertools
import numpy as np
import pandas as pd
from itertools import product
from collections import deque

Node class 

In [221]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

this function creats a full tree based on the levels we need and the specific leafes we want !!!!

In [222]:
def generate_full_tree(level, current_level=1, leaf_values=None):
    """Generate a full binary tree of a given level with specified leaf values."""
    if leaf_values and current_level == level:
        # Use the first value of leaf_values and remove it from the list
        return TreeNode(leaf_values.pop(0))
    
    node_label = 'root' if current_level == 1 else f'c{current_level}'
    node = TreeNode(node_label)
    
    if current_level < level:
        node.left = generate_full_tree(level, current_level + 1, leaf_values)
        node.right = generate_full_tree(level, current_level + 1, leaf_values)
    
    return node

this function will run the the vectur on the tree and return ist prediction for Error calculation 

In [223]:
def run_and_classify(tree, Features_list, label):
    current_node = tree
    for direction in Features_list:
        if direction == 1:
            if current_node.left is not None:
                current_node = current_node.left
        elif direction == 0:
            if current_node.right is not None:
                current_node = current_node.right

    # We should be at a leaf node now
    if current_node.left is None and current_node.right is None:
        return current_node.val == label
    else:
        return False  # The node is not a leaf node

# test run 
# tree = generate_full_tree(3, leaf_values=[0,0,0,0])
# # print_tree(tree)
# check = run_and_classify(tree,[0,1,0,1,1,0,0,0],0)
# print(check)

this function will creat all the leaf options

In [224]:
def generate_leafs_options(n):
    """Generate a list of binary numbers from 0 to n-1."""
    binary_list = []
    for i in range(n):
        # Format the number as a binary string with leading zeros
        binary_str = format(i, f'0{n.bit_length()-1}b')
        binary_list.append(binary_str)
    return binary_list

Print the tree the way we want it 

In [225]:
def print_tree(root):
    """Helper function to print a binary tree in the specified format."""
    lines, *_ = display_aux(root)
    for line in lines:
        print(line)

def display_aux(node):
    """Returns list of strings, width, height, and horizontal coordinate of the root."""
    if node.left is None and node.right is None:
        line = f'{node.val}'
        width = len(line)
        height = 1
        middle = width // 2
        return [line], width, height, middle

    left, n, p, x = display_aux(node.left) if node.left else ([], 0, 0, 0)
    right, m, q, y = display_aux(node.right) if node.right else ([], 0, 0, 0)
    s = f'{node.val}'
    u = len(s)
    first_line = (x + 2) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y + 1) * ' '
    second_line = (x) * ' ' + '1/' + (n - x - 1 + u + y) * ' ' + '\\0' + (m - y - 1) * ' '
    if p < q:
        left += [n * ' '] * (q - p)
    elif q < p:
        right += [m * ' '] * (p - q)
    zipped_lines = zip(left, right)
    lines = [first_line, second_line] + [a + (u + 2) * ' ' + b for a, b in zipped_lines]
    return lines, n + m + u + 2, max(p, q) + 2, n + u // 2

Test run

In [226]:
# k = 3
# leaf_values = "0100"
# leaf_values = list(leaf_values)  # Convert the leaf values string to a list
# tree = generate_full_tree(k, leaf_values=leaf_values)
# print_tree(tree)

Load The Data

In [227]:
def load_data(file_path):
    data = np.loadtxt(file_path, dtype=int)
    X = data[:, :-1]
    y = data[:, -1]
    return X, y

Features, Labels = load_data('vectors.txt')
K = 3 
number_of_options = pow(2,K+1)
options = generate_leafs_options(number_of_options) #[0000,0001,0010,0011,0100,0101, 0110,0111,1000,1001,1010,1011,1100,1101,1110,1111]
print (options)

# print("Features (X):")
# print(X[0])
# print("Labels (y):")
# print(y)

['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']


now what is left to do is for every leaf option creat a tree run all the vecturs on that tree    
self calculate the Error (easy)   
and save the best tree !    

In [228]:
best_tree = None
best_error = None

for leaf_option in options:
    # creat the tree
    list_leaf_option = list(leaf_option)
    tree = generate_full_tree(K, leaf_values=list_leaf_option)
    print_tree(tree)
    Error = 0
    # now for every tree run over all the vecturs
    for features,lable in zip(Features, Labels):
        check = run_and_classify(tree,features,lable)
        if check == False:
            print("faile at featurs: " + str(features) + " and lable: " + str(lable))
            Error = Error + 1

    # now check the error of the tree 
    if best_error == None or best_error > Error:
        best_error = Error
        best_tree = tree

    ___root__     
  1/         \0   
  c2          c2  
1/  \0      1/  \0
0    0      0    0
faile at featurs: [0 1 0 1 1 0 0 0] and lable: 1
faile at featurs: [1 1 0 0 1 1 0 0] and lable: 0
faile at featurs: [0 0 0 0 1 1 1 1] and lable: 1
faile at featurs: [1 1 1 1 1 1 1 0] and lable: 0
faile at featurs: [1 1 0 1 1 0 1 1] and lable: 0
faile at featurs: [0 0 1 1 1 0 0 1] and lable: 1
faile at featurs: [0 0 1 1 0 0 0 0] and lable: 0
faile at featurs: [0 0 1 1 1 1 1 1] and lable: 0
faile at featurs: [1 0 1 1 0 1 1 1] and lable: 0
faile at featurs: [0 0 0 0 1 1 0 0] and lable: 1
faile at featurs: [1 0 0 1 0 1 1 1] and lable: 0
faile at featurs: [1 1 1 0 0 0 0 0] and lable: 1
faile at featurs: [1 1 1 1 0 0 0 0] and lable: 0
faile at featurs: [1 1 1 0 1 0 0 1] and lable: 0
faile at featurs: [0 0 0 1 0 1 0 0] and lable: 0
faile at featurs: [0 1 1 0 0 0 0 1] and lable: 0
faile at featurs: [0 0 1 1 0 0 0 1] and lable: 1
faile at featurs: [1 0 1 0 0 0 0 1] and lable: 0
faile at featurs: [1 1 

In [229]:
print (best_error)

150


In [230]:
print_tree(tree)

    ___root__     
  1/         \0   
  c2          c2  
1/  \0      1/  \0
1    1      1    1
